In [1]:
import glob, os
import pandas as pd
import pdb
from IPython.core import display as ICD
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)


Collect files with bad volumes from our text input file:

In [3]:
# CODE TIP -extend functionally by adding automatic reference to volume check file 

filename='bad_volumes.txt'
vol_dict = {}
with open(filename) as f:
    content=f.readlines()
    for line in content:
        if "training" in line:
            file=line.split("\t")[0]
            vol=line.split("\t")[1].strip()
        else:
            file=line.split("\t\t")[0]
            vol=line.split("\t\t")[1].strip()
        temp_tuple=(file,vol)
        sub_id = file.split(":")[1].split("_")[0]
        if sub_id not in vol_dict:
            vol_dict[sub_id] = [temp_tuple]
        else:
            vol_dict[sub_id].append(temp_tuple)



In [19]:
## setup scan notes dataframe

df_orig = pd.read_csv("w1_notes.csv", encoding='latin 1')
w1_scan_notes = df_orig[['participantID', 'w1scan_scannotes']]
w1_scan_notes=w1_scan_notes.drop(0)
w1_scan_notes.set_index('participantID', inplace=True)
w1_scan_notes.head()

,w1scan_scannotes
participantID,
BBX_001,NaN
bbx_002,NaN
bbx_005,NaN
bbx_004,NaN
bbx_006,NaN


### Below we list out the files with bad volumes and view the scan notes. 
- View [`CHANGELOG.md`]() for further details on scan if available.

*Note we need to add scan notes file input, currently only lists bad volume files*

In [21]:
print("Here we look at files found not matching their expected volume. \
      \nExpectations: { rl: 212, training: 233, resting: 147 } \n")
  
print("Subjects with bad files: \t{}".format(sorted(vol_dict.keys())))
for sub in vol_dict:
    # get information from resources to print specific subject information
    bbx_id= sub.replace("sub-", "bbx_")

    train=[x for x in vol_dict[sub] if "training" in x[0]]
    rl=[x for x in vol_dict[sub] if "rl" in x[0]]
    rest=[x for x in vol_dict[sub] if "resting" in x[0]]
    print("\n________________________________________________________________")
    print("\nID: {}\n".format(sub))
    print("> FILES WITH BAD VOLUMES")
    if train:
        print("\n- Training files: \n")
        for file, vol in train:
            ses=file.split("_")[1]
            print("{} \t\t{}".format(file,vol))
    if rl:
        print("\n- Prediction Error files: \n")
        for file, vol in rl:
            print("{} \t\t{}".format(file,vol))
    if rest:
        print("\n- Resting files: \n")
        for file, vol in rest:
            print("{} \t\t{}".format(file,vol))
            
    print("\n> SCAN NOTES")
    ICD.display(w1_scan_notes.loc[bbx_id])
    print("\n\n")


Here we look at files found not matching their expected volume.       
Expectations: { rl: 212, training: 233, resting: 147 } 

Subjects with bad files: 	['sub-002', 'sub-044', 'sub-094', 'sub-128', 'sub-146', 'sub-159']

________________________________________________________________

ID: sub-094

> FILES WITH BAD VOLUMES

- Training files: 

File:sub-094_ses-1_task-training_run-1_bold.nii.gz  		Volume:5
File:sub-094_ses-1_task-training_run-2_bold.nii.gz  		Volume:5
File:sub-094_ses-1_task-training_run-3_bold.nii.gz  		Volume:3
File:sub-094_ses-1_task-training_run-4_bold.nii.gz  		Volume:2
File:sub-094_ses-2_task-training_run-1_bold.nii.gz  		Volume:4
File:sub-094_ses-2_task-training_run-2_bold.nii.gz  		Volume:2
File:sub-094_ses-2_task-training_run-3_bold.nii.gz  		Volume:1
File:sub-094_ses-2_task-training_run-4_bold.nii.gz  		Volume:2

- Prediction Error files: 

File:sub-094_ses-2_task-rl_run-1_bold.nii.gz  		Volume:1
File:sub-094_ses-2_task-rl_run-2_bold.nii.gz  		Volume:1

- Res

w1scan_scannotes    Consistently moderate twitching motion throughout due to swallowing 
Name: bbx_094, dtype: object





________________________________________________________________

ID: sub-128

> FILES WITH BAD VOLUMES

- Training files: 

File:sub-128_ses-1_task-training_run-1_bold.nii.gz  		Volume:154
File:sub-128_ses-1_task-training_run-2_bold.nii.gz  		Volume:131
File:sub-128_ses-1_task-training_run-3_bold.nii.gz  		Volume:131
File:sub-128_ses-1_task-training_run-4_bold.nii.gz  		Volume:147

- Resting files: 

File:sub-128_ses-1_task-resting_bold.nii.gz  		Volume:103

> SCAN NOTES


w1scan_scannotes    Some movement throughout but not too bad 
Name: bbx_128, dtype: object





________________________________________________________________

ID: sub-159

> FILES WITH BAD VOLUMES

- Resting files: 

File:sub-159_ses-1_task-resting_bold.nii.gz  		Volume:68

> SCAN NOTES


...

w1scan_scannotes    NaN
Name: bbx_002, dtype: object





________________________________________________________________

ID: sub-044

> FILES WITH BAD VOLUMES

- Resting files: 

File:sub-044_ses-2_task-resting_bold.nii.gz  		Volume:122

> SCAN NOTES


w1scan_scannotes    Great job staying still; great scan except for taste disconnect issue in run 2
Name: bbx_044, dtype: object





________________________________________________________________

ID: sub-146

> FILES WITH BAD VOLUMES

- Training files: 

File:sub-146_ses-2_task-training_run-4_bold.nii.gz  		Volume:19

> SCAN NOTES


w1scan_scannotes    NaN
Name: bbx_146, dtype: object

### BIDS Directory Check

In [29]:
sessions=['ses-1']
for sess_id in sessions:
    if "1" in sess_id:
        print("\n> SESSION 1\n")
    else:
        print("\n> SESSION 2\n")
        
    #ses_mia=pd.read_csv('bro_{}_missing.csv'.format(sess_id),sep="\t")
    #ses_mia.set_index("Unnamed: 0", inplace=True)
    #ses_mia.index.names=["subjects"]
    
    ses_pmia=pd.read_csv('qa/bbx_{}_partial_missing.csv'.format(sess_id),sep="\t")
    ses_pmia.set_index("Unnamed: 0", inplace=True)
    ses_pmia.index.names=["subjects"]
    
    
        
    
    #ses_fd=pd.read_csv('bro_{}_found.csv'.format(sess_id),sep="\t")
    #ses_fd.set_index("Unnamed: 0", inplace=True)
    #ses_fd.index.names=["subjects"]
    
    

    #print("All files found missing: \t{}\n".format(' '.join(ses_mia.index.values)))
    print("Some files found missing in subjects: \n")# \t{}\n".format(' '.join(ses_pmia.index.values)))
    #print("All files found: \t\t{}\n".format(' '.join(ses_fd.index.values)))

    #print("\nDataframe (Missing files): ")
    #ICD.display(ses_mia)
    
    for sub in ses_pmia.index.values:
        print("> {} \n".format(sub))
        bro_id = sub.replace("sub-", "bbx_")
        ICD.display(ses_pmia.loc[sub, :])
        ICD.display(w1_scan_notes.loc[bbx_id])
        print("\n")
    print("\nDataframe (Partial files found): ")
    ICD.display(ses_pmia)
    print("\n")
    
    #print("\nDataframe (All files found): ")
    #ICD.display(ses_fd)
    
    


> SESSION 1

Some files found missing in subjects: 

> sub-025 



anat          1
fmap          0
func          0
runs_rest     0
runs_train    0
Name: sub-025, dtype: int64

w1scan_scannotes    NaN
Name: bbx_146, dtype: object



> sub-060 



anat          1
fmap          2
func          2
runs_rest     0
runs_train    2
Name: sub-060, dtype: int64

...



> sub-155 



anat          1
fmap          0
func          0
runs_rest     0
runs_train    0
Name: sub-155, dtype: int64

w1scan_scannotes    NaN
Name: bbx_146, dtype: object




Dataframe (Partial files found): 


,anat,fmap,func,runs_rest,runs_train
subjects,,,,,
sub-025,1,0,0,0,0
sub-060,1,2,2,0,2
sub-104,1,0,0,0,0
sub-115,1,0,0,0,0
sub-117,1,0,0,0,0
sub-134,1,0,0,0,0
sub-155,1,0,0,0,0
